# Day 09. Exercise 03
# Ensembles

## 0. Imports

In [113]:
import pandas as pd
import numpy as np
import joblib

In [114]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, KFold
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

## 1. Preprocessing

1. Создайте тот же кадр данных, что и в предыдущем упражнении.
2. Используя `train_test_split` с параметрами `test_size=0.2`, `random_state=21` получите `X_train`, `y_train`, `X_test`, `y_test`, а затем получите `x_train`, `y_train`, `X_valid`, `y_valid` из предыдущих `x_train`, `y_train`. Используйте дополнительный параметр `stratify`.

In [115]:
df = pd.read_csv('../data/day-of-week-not-scaled.csv')
df1 = pd.read_csv('../data/dayofweek.csv')
df['dayofweek'] = df1['dayofweek']

x = df.drop(['dayofweek'], axis=1)
y = df['dayofweek']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=21, stratify=y)

## 2. Individual classifiers

1. Снова обучите SVM, дерево решений и случайный лес с наилучшими параметрами, которые вы получили в упражнении 01, с `random_state=21` для всех них.
2. Оцените `точность`, `точность` и `вызов` для них на проверочном множестве.
3. Результат в каждой ячейке раздела должен выглядеть следующим образом:

```
accuracy is 0.87778
precision is 0.88162
recall is 0.87778
```

In [116]:
best_params = {'random_state': 21,'probability': True, 'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}
svc = SVC(**best_params)
svc.fit(x_train, y_train)
y_pred = svc.predict(x_test)

print(f"accuracy is {accuracy_score(y_test, y_pred):.5f}")
print(f"precision is {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, y_pred, average='weighted'):.5f}")

accuracy is 0.88757
precision is 0.89267
recall is 0.88757


In [117]:
best_params  = {'random_state': 21, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': np.int64(22)}
dtc = DecisionTreeClassifier(**best_params)
dtc.fit(x_train, y_train)
y_pred = dtc.predict(x_test)

print(f"accuracy is {accuracy_score(y_test, y_pred):.5f}")
print(f"precision is {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, y_pred, average='weighted'):.5f}")

accuracy is 0.89053
precision is 0.89262
recall is 0.89053


In [118]:
best_params = {'random_state': 21, 'class_weight': None, 'criterion': 'gini', 'max_depth': np.int64(28), 'n_estimators': 50}
rfc = RandomForestClassifier(**best_params)
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)

print(f"accuracy is {accuracy_score(y_test, y_pred):.5f}")
print(f"precision is {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, y_pred, average='weighted'):.5f}")

accuracy is 0.92899
precision is 0.93009
recall is 0.92899


## 3. Voting classifiers

1. Используя `VotingClassifier` и три модели, которые вы только что обучили, рассчитайте `точность`, `точность` и `вызов` на проверочном множестве.
2. Поиграйте с другими параметрами.
3. Рассчитайте `точность`, `точность` и `вызов` на тестовом наборе для модели с наилучшими весами с точки зрения точности (если их несколько с одинаковыми значениями, выберите ту, у которой выше точность).

In [119]:
model = VotingClassifier(estimators=[('svc', svc), ('dtc', dtc), ('rfc', rfc)])
param_grid = {'voting': ['soft', 'hard'],
              'weights': [[1, 2, 3], [3, 2, 1], [1, 1, 1]]}

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=21)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=skf, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

print(f'Best params: {grid_search.best_params_}')
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred):.5f}')

print(f"accuracy is {accuracy_score(y_test, y_pred):.5f}")
print(f"precision is {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, y_pred, average='weighted'):.5f}")

Best params: {'voting': 'hard', 'weights': [1, 2, 3]}
Accuracy: 0.92604
accuracy is 0.92604
precision is 0.92740
recall is 0.92604


## 4. Bagging classifiers

1. Используя `BaggingClassifier` и `SVM` с наилучшими параметрами, создайте ансамбль, попробуйте разные значения `n_estimators`, используйте `random_state=21`.
2. Поиграйте с другими параметрами.
3. Рассчитайте `точность`, `точность` и `вызов` для модели с лучшими параметрами (по точности) на тестовом наборе (если их несколько с одинаковыми значениями, выберите ту, у которой выше точность).

In [120]:
best_params = {'random_state': 21,'probability': True, 'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}
svc = SVC(**best_params)
model = BaggingClassifier(estimator=svc, random_state=21)

param_grid = {'n_estimators': [10, 20],
              'bootstrap': [True, False],
              'bootstrap_features': [True, False]  }

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=21)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=skf, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

print(f'Best params: {grid_search.best_params_}')
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred):.5f}')

print(f"accuracy is {accuracy_score(y_test, y_pred):.5f}")
print(f"precision is {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, y_pred, average='weighted'):.5f}")

Best params: {'bootstrap': True, 'bootstrap_features': False, 'n_estimators': 20}
Accuracy: 0.89941
accuracy is 0.89941
precision is 0.90369
recall is 0.89941


## 5. Stacking classifiers

1. Для достижения воспроизводимости в этом случае вам придется создать объект генератора кросс-валидации: `StratifiedKFold(n_splits=n, shuffle=True, random_state=21)`, где `n` вы попытаетесь оптимизировать (подробности ниже).
2. Используя `StackingClassifier` и три модели, которые вы недавно обучили, рассчитайте `точность`, `точность` и `результативность` на валидационном множестве, попробуйте разные значения `n_splits` `[2, 3, 4, 5, 6, 7]` в генераторе кросс-валидации и параметра `passthrough` в самом классификаторе,
3. Вычислите `точность`, `точность` и `вызов` для модели с наилучшими параметрами (по точности) на тестовом множестве (если их несколько с одинаковыми значениями, выберите ту, у которой выше точность). Используйте `final_estimator=LogisticRegression(solver='liblinear')`.

In [121]:
model = StackingClassifier(estimators=[('svc', svc), ('dtc', dtc), ('rfc', rfc)],
                           final_estimator=LogisticRegression(solver='liblinear', random_state=21))

param_grid = {'cv': [StratifiedKFold(n_splits=n, shuffle=True, random_state=21) for n in range(2, 12, 2)],
              'stack_method': ['auto', 'predict_proba', 'predict'],
              'passthrough': [True, False]}

gscv = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', n_jobs=-1)

gscv.fit(x_train, y_train)
print(f'Best params: {gscv.best_params_}')
best_model = gscv.best_estimator_
y_pred = best_model.predict(x_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.5f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.5f}")

Best params: {'cv': StratifiedKFold(n_splits=4, random_state=21, shuffle=True), 'passthrough': True, 'stack_method': 'auto'}
Accuracy: 0.92604
Precision: 0.92709
Recall: 0.92604


## 6. Predictions

1. Выберите лучшую модель по точности (если их несколько с одинаковыми значениями, выберите ту, у которой точность выше).
2. Проанализируйте: для какого дня недели ваша модель делает больше всего ошибок (в % от общего числа образцов этого класса в вашем полном наборе данных), для какого имени лаборатории и для каких пользователей.
3. Сохраните модель.

In [122]:
def error_rate(cm, day_i):
    cm_df = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
    cm_df = pd.DataFrame(cm_df, index=np.unique(y_test), columns=np.unique(y_test))

    if cm:
        print(f'Confusion Matrix:\n{cm_df}')

    print(f'Error rates:')
    if len(day_i) == 0:
        day_i = cm_df.index

    for i in day_i:
        day = (cm_df.loc[i].sum() - cm_df.loc[i, i]) / cm_df.loc[i].sum() * 100
        print(f'day_{i}: Error Rate: {day:.5f} %')

def day_error(day_i):
    print(f'Day errors:')
    y_t_p = pd.DataFrame({'index': y_test.index,'y_test': y_test, 'y_pred': y_pred})

    if len(day_i) == 0:
        day_i = np.unique(y_test)

    for i in day_i:
        yy = y_t_p[y_t_p['y_test'] == i]
        yy = yy[yy['y_test'] != yy['y_pred']]

        for j in range(len(yy)):
            s = df.loc[yy.iloc[j, 0]]
            s = s[s != 0]
            print(f'True dayofweek: {i} | Pred dayofweek: {yy.iloc[j, 2]} | User UID: {s.index[2]} | Project Name: {s.index[3]}')

In [123]:
error_rate(True, [])
day_error([0])

Confusion Matrix:
    0   1   2   3   4   5   6
0  22   0   1   0   1   1   2
1   1  49   1   3   0   1   0
2   0   0  28   2   0   0   0
3   1   1   0  76   0   1   1
4   0   0   0   0  19   2   0
5   0   0   0   2   0  49   3
6   0   0   0   0   0   1  70
Error rates:
day_0: Error Rate: 18.51852 %
day_1: Error Rate: 10.90909 %
day_2: Error Rate: 6.66667 %
day_3: Error Rate: 5.00000 %
day_4: Error Rate: 9.52381 %
day_5: Error Rate: 9.25926 %
day_6: Error Rate: 1.40845 %
Day errors:
True dayofweek: 0 | Pred dayofweek: 2 | User UID: uid_user_4 | Project Name: labname_laba06s
True dayofweek: 0 | Pred dayofweek: 4 | User UID: uid_user_4 | Project Name: labname_project1
True dayofweek: 0 | Pred dayofweek: 5 | User UID: uid_user_31 | Project Name: labname_project1
True dayofweek: 0 | Pred dayofweek: 6 | User UID: uid_user_6 | Project Name: labname_laba06
True dayofweek: 0 | Pred dayofweek: 6 | User UID: uid_user_14 | Project Name: labname_code_rvw
